In [1]:
# change working dir
import os
os.chdir('../')

import torch
from src.datasets.scannet import ScanNetDataset
from src.threedsam.threedsam_modules.epipolar_attention import EpipolarAttention, CrossAttention

root_dir = '/media/morgen/T7/dataset/scannet_train'
npz_path = '/media/morgen/T7/dataset/scannet_indices/scene_data/train/scene0066_00.npz'
intrinsic_path = '/media/morgen/T7/dataset/scannet_indices/intrinsics.npz'
dpt_weight_path = '/home/morgen/projects/3DSAM/weights/dpt/dpt_hybrid_nyu-2ce69ec7.pt'
dataset = ScanNetDataset(root_dir, npz_path, intrinsic_path, dpt_weight_path)

/home/morgen/.conda/envs/3dsam/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/morgen/.conda/envs/3dsam/lib/python3.8/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [2]:
# load a sample
sample = dataset[200]
K0 = sample['K0']
K1 = sample['K1']
R = sample['T_0to1'][0:3, 0:3]
t = sample['T_0to1'][0:3, [-1]]

K0.shape

torch.Size([3, 3])

In [3]:
torch.cuda.is_available()

True

In [4]:
config = {
    'd_model': 512,
    'nhead': 8,
    'area_width': 5
}

epi_info = {
    'h0c': 30,
    'w0c': 40,
    'K0': K0.unsqueeze(dim=0).cuda(),
    'K1': K1.unsqueeze(dim=0).cuda(),
    'R': R.unsqueeze(dim=0).cuda(),
    't': t.unsqueeze(dim=0).cuda(),
    'scale': 16,
}

feat0 = torch.randn((1, 1200, 512)).cuda()
feat1 = torch.randn((1, 1200, 512)).cuda()
epi_attention = EpipolarAttention(config)
epi_attention.to('cuda')

# 806MB

EpipolarAttention(
  (layer): CrossAttention(
    (q_proj): Linear(in_features=512, out_features=512, bias=False)
    (k_proj): Linear(in_features=512, out_features=512, bias=False)
    (v_proj): Linear(in_features=512, out_features=512, bias=False)
    (attention): One2ManyAttention()
    (merge): Linear(in_features=512, out_features=512, bias=False)
    (mlp): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=False)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=1024, out_features=512, bias=False)
    )
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [5]:
for itr in range(4):
    feat0, feat1 = epi_attention(feat0, feat1, epi_info)

True